# The Anatomy of a Python Class

> Author: Samuel Farrens (<samuel.farrens@cea.fr>)  
> Year: 2019

Classes are one of the fundamental building blocks of Python and are essential for object-oriented programming.

In this notebook we will explore how classes work and look at tips and tricks for getting the most out of them. By the end of this tutorial you should have, not only a much better understanding of Python classes, but also some new ideas for writing better code.

## Contents

---

* [1 Disecting a Class](#1-Disecting-a-Class)
  * [The Class Dictionary](#The-Class-Dictionary)
  * [Instantiation](#Instantiation)
* [2 Methods](#2-Methods)
  * [Instance Methods](#Instance-Methods)
  * [Static Methdos](#Static-Methods)
  * [Class Methods](#Class-Methods)
  * [Abstract Methods](#Abstract-Methods)
* [3 Inheritance](#3-Inheritance)
* [4 Composition](#4-Composition)

## 1 Disecting a Class

---

To get a better grip on what a class is and how it works lets have a look at what is going on "inside".

We can start disecting a class by looking at its dictionary.

<br/>

### The Class Dictionary

Classes are defined with the keyword `class`, much like the keyword `def` is used for defining functions. In the following cell we define a class without any attributes (note the use of the null `pass` statement).

In [1]:
# Define a dummy class
class myClass:
    pass

This object clearly has little use, but it is a good starting place to look at the structure of classes in general. We can start by looking at the special `__dict__` attribute of the object.

In [2]:
# Print the class dictionary
print(myClass.__dict__)

{'__module__': '__main__', '__dict__': <attribute '__dict__' of 'myClass' objects>, '__weakref__': <attribute '__weakref__' of 'myClass' objects>, '__doc__': None}


Unsurpsingly, this attribute is a dictionary, the contents of which are the following:

* `'__doc__'` : The class docstring (defaults to `None`).
* `'__module__'` : The name of module in which the class was defined.
* `'__weakref__'` : A special attribute that allows *weak referencing*.
* `'__dict__'` : A special attribute the defines the dictionary object. 

Now, let's see what happens when we assign a new attribute to the class. Class attributes are accessed and set using a dot (`.`) following the class name.

> Note that any Python object can be assigned as a class attribute. We will come back to this concept later!

In [3]:
# Assign the attribute myattr with value True to the class
myClass.myattr = True

# Print the class dictionary
print(myClass.__dict__)

{'__module__': '__main__', '__dict__': <attribute '__dict__' of 'myClass' objects>, '__weakref__': <attribute '__weakref__' of 'myClass' objects>, '__doc__': None, 'myattr': True}


We can see that the class dictionary has a new entry with the key `myattr` and corresponding value `True`. We can even demonstrate that this dictionary behaves as any other...

In [4]:
# Print the class dictionary with key 'myattr'
print(myClass.__dict__['myattr'])

True


...except that it does not allow assignment.

In [5]:
# Assign a new value to 'myattr' in the class dictionary
myClass.__dict__['myattr'] = False

TypeError: 'mappingproxy' object does not support item assignment

Now we can define a new class with some predefined attributes and see what changes.

In [6]:
# Define a new dummy class with some attributes
class myClass2:
    """ This is my class 
    """

    mybool = True
    myint = 1
    myfloat = 1.0
    mystring = 'string'
    
    def myfunc():
        pass
    
# Print the class dictionary    
print(myClass2.__dict__)

{'__module__': '__main__', '__doc__': ' This is my class \n    ', 'mybool': True, 'myint': 1, 'myfloat': 1.0, 'mystring': 'string', 'myfunc': <function myClass2.myfunc at 0x1111b0ae8>, '__dict__': <attribute '__dict__' of 'myClass2' objects>, '__weakref__': <attribute '__weakref__' of 'myClass2' objects>}


All of the attributes can be found in the class dictionary, including the dummy method `myfunc`. Notice that `__doc__` is now a string. 

All of these attributes can be accessed...

In [18]:
# Print the class attributes
print('mybool =', myClass2.mybool)
print('myint =', myClass2.myint)
print('myfloat =', myClass2.myfloat)
print('mystring =', myClass2.mystring)
print('myfunc =', myClass2.myfunc)
print('mydoc =', myClass2.__doc__)

mybool = True
myint = 1
myfloat = 1.0
mystring = my new string
myfunc = <function myClass2.myfunc at 0x1111b0ae8>
mydoc =  This is my class 
    


...and modified.

In [17]:
# Modify the value of the string attribute
myClass2.mystring = 'my new string'

# Print the string attribute
print(myClass2.mystring)

my new string


In summary the class `__dict__` attribute is useful for understanding how class attributes are stored, but should really be used directly.

<br/>

### Instantiation

While accessing class attributes directly can be useful, in most applications it will be necessary to create a *class instance*. Every time a class is isntantiated a unique Python class object is created. These objects retain the global class attributes but can also be assigned unique instance attributes.

We can create a class instance by calling the class name followed by `()`.

In [7]:
# Create an instance of the class
myinst = myClass2()

# Print the instance dictionary
print(myinst.__dict__)

{}


Here we see that the instance dictionary is empty, which simply means that no *instance attributes* have been set. If we look at the instance class dictionary we can see all of the class attributes.

In [8]:
# Print the instance class dictionary
print(myinst.__class__.__dict__)

{'__module__': '__main__', '__doc__': ' This is my class \n    ', 'mybool': True, 'myint': 1, 'myfloat': 1.0, 'mystring': 'string', 'myfunc': <function myClass2.myfunc at 0x1111b0ae8>, '__dict__': <attribute '__dict__' of 'myClass2' objects>, '__weakref__': <attribute '__weakref__' of 'myClass2' objects>}


In order to assign instance attributes we will need to define a special initialisation method (`__init__`), which takes the special variable `self` as an argument.

In [31]:
# Define a new dummy class with an init method
class myClass3:
    
    def __init__(self):
        
        self.myfloat = 3.5
   
# Create an instance of the class and print the attribute properties
myinst = myClass3()
print(myinst.__dict__)
print('myfloat =', myinst.myfloat)


{'myfloat': 3.5}
myfloat = 3.5


`self` represents the any given class instance and attributes are assigned to it as usual with a dot. Printing the instance dictionary (which is equivalent to `self.__dict__`) reveals the attributes that have been assigned to it.

As with class attributes, instance attribute values can be modified.

In [28]:
myinst.myfloat = 7.2
print(myinst.__dict__)
print('myfloat =', myinst.myfloat)

{'myfloat': 7.2}
myfloat = 7.2


Additional agruments can be passed to `__init__` allowing for a more instance specific initialisation of the class.

In [30]:
# Define a new dummy class with a more dynamic init method
class myClass4:
    
    def __init__(self, value):
        
        self.myfloat = value

# Create instances of the class and print the attribute properties
myinst1 = myClass4(3.14)
myinst2 = myClass4(6.28)
print('myfloat =', myinst1.myfloat)
print('myfloat =', myinst2.myfloat)

myfloat = 3.14
myfloat = 6.28


When designing a class it is important to think about which attributes need to be assigned to the class and which to the class instances.

In the following section we will focus on the various types of method attributes.

## 2 Methods

---

<br/>

### Instance Methods

In the previous section you learned how to instantiate a class and that any Python object can be assigned as a class attribute. As functions are Python objects, they too be assigned to class attributes.

In [36]:
class myClass5:
    
    def __init__(self):
        pass
    
def myfunc(a, b):
    
    return a + b

myinst = myClass5()
myinst.add = myfunc

print(myinst.__dict__)
print('1 + 2 =', myinst.add(1, 2))

{'add': <function myfunc at 0x1111b0c80>}
1 + 2 = 3


We can see that the function `myfunc` has been assigned to the instance attribute `add`. This, however, is not a very efficient way to assign methods to a class.

> Note that functions are called methods when they are class attributes.

Let's look at a better way of defining methods for class instances.

In [48]:
class Einstein:
    
    def __init__(self):
        
        self.c = 3e8
    
    def energy(self, mass):
        
        return mass * self.c ** 2
    
    def mass(self, energy):
        
        return energy / (self.c ** 2)

myinst = Einstein()
print(myinst.__dict__)        
print('E = {} J'.format(myinst.energy(80)))
print('m = {} Kg'.format(myinst.mass(9e16)))

{'c': 300000000.0}
E = 7.2e+18 J
m = 1.0 Kg


In this example we have not assigned the method `energy` to the instance, rather we have passed the instance to the method. This enables the method to see all instance attributes (in this case `c`). This is an extremely powerful mechanism in Python. Instead of defining two independent functions for calculating mass and energy, each of which would have had to hardwire the value of the speed of light or take it as an argument, we define it once as an instance attribute.

### Static Methods

In [ ]:
class Calculator:
    
    def add(x, y):
        return x + y
    
    def substract(x, y):
        return x - y

    def multiply(x, y):
        return x * y
    
    def divide(x, y):
        return x / y

In [ ]:
print('1 + 2 =', Calculator.add(1, 2))
print('5 - 2 =', Calculator.substract(5, 2))
print('7 * 4 =', Calculator.multiply(7, 4))
print('6 / 2 =', Calculator.divide(6, 2))

In [ ]:
calc = Calculator()
print('1 + 2 =', calc.add(1, 2))

In [ ]:
class staticCalculator:
    
    @staticmethod
    def add(x, y):
        return x + y
    
    @staticmethod
    def substract(x, y):
        return x - y
    
    @staticmethod
    def multiply(x, y):
        return x * y
    
    @staticmethod
    def divide(x, y):
        return x / y

In [ ]:
calc = staticCalculator()
print('1 + 2 =', staticCalculator.add(1, 2))
print('1 + 2 =', calc.add(1, 2))

### Class Methods

In [ ]:
from math import pi, sqrt

class StefBoltz:
    """ The Stefan–Boltzmann law
    """
    
    sigma = 5.670367e-8 # Wm^−2K^−4
        
    @classmethod
    def luminosity(cls, radius, effective_temp):
     
        return 4 * pi * radius ** 2 * cls.sigma * effective_temp ** 4
    
    @classmethod
    def radius(cls, luminosity, effective_temp):
    
        return sqrt(luminosity / (4 * pi * cls.sigma * effective_temp ** 4))
    
    @classmethod
    def temperature(cls, radius, luminosity):
    
        return (luminosity / (4 * pi * radius ** 2 * cls.sigma)) ** 0.25

In [ ]:
print('The luminosity of the Sun is {:.2e}W'.format(StefBoltz.luminosity(7e8, 5800)))
print('The radius of Alpha Centauri is {:.2e}m'.format(StefBoltz.radius(4e28, 9700)))
print('The effective temperature of the Earth is {:.2f}K'
      ''.format(StefBoltz.temperature(6.37e6, 1.75e+17)))

### Abstract Methods

## 3 Inheritance

---

## 4 Composition

---